#Two-lane comparison

In [19]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss
from datetime import datetime
import matplotlib.patches as patches

#Parameters
N =[10,20,30,40,50,60,70,80,90] #amount of vehicles (density)
pt = 1 #unit time (step)
s = 100 #number of simulations
t = 500 #time that the each simulation longs
vmax = 1 #speed limit
vmin= 0 #speed limit
tM= 50 #capacity for each lane

class Vehiculo:
    id_iter = itertools.count() #to generate an id for each agent
    def __init__(self, tipo, posx, velocidad,posy):
        self.id = next(Vehiculo.id_iter) #id
        self.t = tipo
        self.x = posx #X coordinate for location
        self.y = posy #Y coordinate
        self.vm = velocidad
        self.d = 0 #traveled distance
        self.a = 0.01 #aceleration default value
        self.m = random.choice(["vmax", "vm"]) #tag for following distance behavior, change the options considering the experiment you want to simulate
        self.c = random.choice(["vmax", "vm"]) #tag for lane change behavior, change the options considering the experiment you want to simulate
        self.f = vmax

    def mover(self,acel):
        self.a = acel
        #vmax = 1+0.1*random.choice([-1,0,1]) #uncomment for the experiment of different speed limits
        self.vm = max(vmin, min(vmax, self.vm + self.a*pt))
        self.d =self.vm*pt

class Carril:
    def __init__(self, posy):
        self.y = posy

def parametros(v,vm):
  distancias = {
      ("vmax"): vmax, #change it with 1 for the experiment of different speed limits, otherwise :vmax
      ("vm"): vm}
  return distancias.get((v),"NA")

def iniciar(n): #to create agents of the model
    global vehiculos, carriles, filenames

    filenames = []
    vehiculos=[]

    #to create lanes
    carriles = []
    c1 = Carril(1.5)
    carriles.append(c1)
    c2 = Carril(2)
    carriles.append(c2)

    #to create vehicles

    while len(vehiculos) < n:
        v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4), random.choice([c.y for c in carriles]))#,"vm")
        if round(v.x,0) not in [round(y.x,0) for y in vehiculos if v.y == y.y]:
            vehiculos.append(v)

def toroide(): #toroidal space
    for v in vehiculos:
        if v.x > tM:
            v.x = v.x % tM

def duplicar(): #to temporarily create a vehicle so the last vehicle in a lane can "see" the first one of the same lane.
    global a
    for c in carriles:
      if len([v.x for v in vehiculos if v.y == c.y])!= 0:
        for v in vehiculos:
          if v.x == min([v.x for v in vehiculos if v.y == c.y]):
            a = v.vm
            copia = Vehiculo("doble",min([v.x for v in vehiculos if v.y == c.y])+tM, a,c.y)
            vehiculos.append(copia)
      else:
          pass

def movimiento(): #to move vehicles along both lanes
  for c in carriles:
    vehiculos.sort(key = lambda v: v.x)
    for v,z in zip([v for v in vehiculos if v.y==c.y],[z for z in vehiculos if z.y==c.y][1:]):
      d = z.x - v.x #distance between vehicles
      v.f = parametros(v.m,(min(1,max(v.vm + 0.011,0.1))))
      if  d >= v.f*pt:
        v.mover(0.01+0.001*random.uniform(-1,1))
      else:
        v.vm = z.vm #to take the speed of the vehicle in front
        v.mover(-0.02) #to brake
        v.f = parametros(v.c, (min(1,max(v.vm + 0.011,0.1))))
        cruce(v,v.f,1)
      v.x = v.d + v.x


def cruce(v,fun,p): #for lane changing
    lmax=v.x+vmax
    lmin=v.x-fun
    for c in carriles:
      if len([z for z in vehiculos if z.y != v.y and z.x < lmax and z.x > lmin and abs(v.y-z.y)==abs(v.y-c.y)])==0 and v.y!=c.y:
        v.y = v.y - (v.y-c.y)
      else:
        pass

def borrar(): #to eliminate the temporarily vehicle
    for c in carriles:
        for v in vehiculos:
            if v.t == "doble":
                vehiculos.remove(v)

def imitacion(): #imitation behavior
  alrededor=[]
  for v in vehiculos:
    for z in vehiculos:
      if v!=z:
        if abs(v.x - z.x) <= 1.1:
          alrededor.append(z.m)
    if len(alrededor)!=0:
      if len([i for i in alrededor if i == "vmax"])>len([i for i in alrededor if i == "vm"]):
        v.m="vmax"
      elif len([i for i in alrededor if i == "vmax"])<len([i for i in alrededor if i == "vm"]):
        v.m="vm"
      else:
        pass
    alrededor=[]

def interaccion(n,s): #all functions are called here
    for i in range(1,t+1):
      duplicar()
      #imitacion() #uncomment if needed
      movimiento()
      borrar()
      toroide()

      #Collected data from each step to generate graphics, we can change what we collect considering the experiment
      vel1.append([1,st.mean([v.vm for v in vehiculos]),n,s])
      med1.append([1,st.median([v.vm for v in vehiculos]),n,s])
      des1.append([1,st.stdev([v.vm for v in vehiculos]),n,s])
      err1.append([1,ss.sem([v.vm for v in vehiculos]),n,s])

      #Uncomment this part to create an image of each step and save them to be able to create a GIF:
      '''
      ig, ax = plt.subplots(figsize=(18,7))
      #carros
      #plot([v.x for v in vehiculos if v.m=="vmax"], [v.y for v in vehiculos if v.m=="vmax"] ,'o', color="darkorange", markersize=3)
      #plot([v.x for v in vehiculos if v.m=="vm"], [v.y for v in vehiculos if v.m=="vm"] ,'o', color="blue", markersize=3)
      #carriles
      ax.add_patch(patches.Rectangle((0, 0.8),50,0.4,edgecolor = 'gray',facecolor = 'gray',fill=True))
      text(1,0.17,"Paso:")
      text(2,0.2,i)
      tick_params(labelcolor='w')
      #grid(b=True, axis='both', color = "white")
      axis([0,tM,0.5,1.5])

      filename = f'paso{i}.png'
      filenames.append(filename)
      plt.savefig(filename)
      plt.close()
      #'''
      '''
      print(["Tiempo:",i])
      print(len([v for v in vehiculos if v.m=="vmax"]))
      print(len([v for v in vehiculos if v.m=="vm"]))
      '''




In [ ]:
#lists
vel1 = []
med1 = []
des1 = []
err1 = []


for i in N:
  for j in range (1,s+1): #the S is the number of simulations
    iniciar(i)
    interaccion(i,j)
  media=[]
  media.append(st.mean([l[1] for l in vel1 if l[2]==i]))
  print(media)
  print("-----------------------------","Acabó N =",i,"-----------------------------")

In [ ]:
#If the images were created, we can generate the GIF
gif_name= "Los 4 comportamientos"

with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        i = imageio.imread(filename)
        writer.append_data(i)

#This eliminates all the created images
for filename in set(filenames):
    os.remove(filename)

#Variation of behavior

In [14]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss

#Parameters
N =[10,20,30,40,50,60,70,80,90]
amax= 0.01
amin = -0.02
pt = 1
s = 100
t = 500
vmax = 1
vmin= 0
tM= 50

class Vehiculo:
    id_iter = itertools.count()
    def __init__(self, tipo, posx, velocidad,posy):
        self.id = next(Vehiculo.id_iter) #id
        self.t = tipo
        self.x = posx
        self.y = posy
        self.vm = velocidad
        self.d = 0
        self.a = 0.01 #default value
        self.f = vmax
        self.m = random.choice(["vmax"])
        self.c = random.choice(["vmax"])
        self.w = "A" #to tag the vehicles that don't change its behavior

    def mover(self,acel):
        self.a = acel
        self.vm = max(vmin, min(vmax, self.vm + self.a*pt))
        self.d =self.vm*pt

class Carril:
    def __init__(self, posy):
        self.y = posy

def parametros(v,vm):
  distancias = {
      ("vmax"): vmax,
      ("vm"): vm}
  return distancias.get((v),"NA")


def iniciar(n):
    global vehiculos, carriles, filenames


    filenames = []

    #lanes
    carriles = []
    c1 = Carril(1)
    carriles.append(c1)
    c2 = Carril(2)
    carriles.append(c2)

    #vehicles
    vehiculos = []

    vi = Vehiculo("carro", round(np.random.uniform(0,tM),0),round(np.random.uniform(0.1,0.3),4), random.choice([c.y for c in carriles]))
    vehiculos.append(vi)

    while len(vehiculos) < n:
        v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4), random.choice([c.y for c in carriles]))
        if round(v.x,0) not in [round(y.x,0) for y in vehiculos if v.y == y.y]:
            vehiculos.append(v)

def toroide():
    for v in vehiculos:
        if v.x > tM:
            v.x = v.x % tM

def duplicar():
    global a
    for c in carriles:
        if len([v.x for v in vehiculos if v.y == c.y])!= 0:
          for v in vehiculos:
              if v.x == min([v.x for v in vehiculos if v.y == c.y]):
                a = v.vm
                copia = Vehiculo("doble",min([v.x for v in vehiculos if v.y == c.y])+tM, a,c.y)
                vehiculos.append(copia)
        else:
            pass

def movimiento():
  for c in carriles:
    vehiculos.sort(key = lambda v: v.x)
    for v,z in zip([v for v in vehiculos if v.y==c.y],[z for z in vehiculos if z.y==c.y][1:]):
      d = z.x - v.x #distancia entre carros
      if  d >= v.f*pt:
        if v.w == "B":
            v.mover(0.02+0.002*random.uniform(-1,1))
        else:
            v.mover(0.01+0.001*random.uniform(-1,1))
      else:
          v.vm = z.vm #con esto se toma la velocidad del siguiente carro
          v.mover(-0.02) # y con esto se frena
          cruce(v,v.f)
      v.x = v.d + v.x

def cruce(v,fun):
    lmax=v.x+vemax
    lmin=v.x-fun
    for c in carriles:
      if len([z for z in vehiculos if z.y != v.y and z.x < lmax and z.x > lmin and abs(v.y-z.y)==abs(v.y-c.y)])==0 and v.y!=c.y:
        v.y = v.y - (v.y-c.y)
      else:
        pass

def borrar():
    for c in carriles:
        for v in vehiculos:
            if v.t == "doble":
                vehiculos.remove(v)

def cambiodefreno(): #to change the behavior of the vehicle
    for c in carriles:
      vehiculos.sort(key = lambda v: v.x)
      for v,z in zip([v for v in vehiculos if v.y==c.y],[z for z in vehiculos if z.y==c.y][1:]): #esto es para saber la posición del que está adelante del carro en el mismo carril
        lmax2=z.x #se considera a la posición del de adelante como el límite superior del rango a considerar
        lmin2=v.x #se considera a la posición del carro que no "quiere" que se le meta otro carro como el límite inferior del rango
        if len([w for w in vehiculos if w.y != v.y and w.x < lmax2 and w.x > lmin2])!=0: #acá se evalúa si hay otro carro en otro carril en el rango definido
          v.f = (min(1,max(v.vm + 0.022,0.1))) #si hay, cambia el parámetro que considera cuando frena y cambi de carril a v.vm (todos empiezan con vmax)
          v.w = "B"
        else:
          v.f = vmax
          v.w = "A"


def interaccion(n,s):
    global u
    for i in range(1,t+1):
        duplicar()
        cambiodefreno()
        movimiento()
        borrar()
        toroide()

        #Información para gráficos por simulación
        vel1.append([1,st.mean([v.vm for v in vehiculos]),n,s])
        med1.append([1,st.median([v.vm for v in vehiculos]),n,s])
        des1.append([1,st.stdev([v.vm for v in vehiculos]),n,s])
        err1.append([1,ss.sem([v.vm for v in vehiculos]),n,s])

        #gif
        '''
        plt.figure(figsize=(20,5))
        plot([v.x for v in vehiculos], [v.y for v in vehiculos] ,'bs')
        plot([v.x for v in vehiculos if v.id == 3], [v.y for v in vehiculos if v.id == 3] ,'ro', markersize=8)
        plot([v.x for v in vehiculos if v.id == 23], [v.y for v in vehiculos if v.id == 23] ,'go', markersize=8)
        for u in [v.id for v in vehiculos]:
          plt.annotate(' '.join(map(str, [round(v.f,1) for v in vehiculos if v.id == u])),
                       (float(' '.join(map(str, [v.x for v in vehiculos if v.id == u]))),float(' '.join(map(str, [v.y for v in vehiculos if v.id == u])))),
                       textcoords="offset points", xytext=(0,5), ha='center', fontsize=6.5)
        #u = float(' '.join(map(str, [v.x for v in vehiculos if v.id == 3])))
        #w = float(' '.join(map(str, [v.y for v in vehiculos if v.id == 3])))
        #plt.annotate(' '.join(map(str, [round(v.f,2) for v in vehiculos if v.id == 3])), (float(' '.join(map(str, [v.x for v in vehiculos if v.id == 3]))),loat(' '.join(map(str, [v.y for v in vehiculos if v.id == 3])))))
        #plt.annotate(' '.join(map(str, [round(v.f,2) for v in vehiculos if v.id == 11])), (float(' '.join(map(str, [v.x for v in vehiculos if v.id == 11]))),loat(' '.join(map(str, [v.y for v in vehiculos if v.id == 11])))))
        text(tM/50,1.2,"Paso:")
        text(tM/20,1.2,i)
        axis([0,tM,0.5,2.5])

        filename = f'paso{i}.png'
        filenames.append(filename)
        plt.savefig(filename)
        plt.close()'''



In [ ]:
#list
vel1 = []
med1 = []
des1 = []
err1 = []

for i in N:
  for j in range (1,s+1): #s is the number of times that the simulation is iterated
    iniciar(i)
    interaccion(i,j)
  media=[]
  media.append(st.mean([l[1] for l in vel1 if l[2]==i]))
  print(media)
  print("-----------------------------","Acabó N =",i,"-----------------------------")

In [ ]:
#If the images were created, we can generate the GIF
gif_name= "Los 4 comportamientos"

with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        i = imageio.imread(filename)
        writer.append_data(i)

#This eliminates all the created images
for filename in set(filenames):
    os.remove(filename)

#Suddenly stopped and road obstruction on the road

In [26]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss
from datetime import datetime
import matplotlib.patches as patches

#Parameters
N =[10,20,30,40,50,60,70,80,90] #amount of vehicles (density)
pt = 1 #unit time (step)
s = 100 #number of simulations
t = 500 #time that the each simulation longs
vmax = 1 #speed limit
vmin= 0 #speed limit
tM= 50 #capacity for each lane
Por =[0.2,0.5,0.8,1] #percentage of stopped vehicles

class Vehiculo:
    id_iter = itertools.count() #to generate an id for each agent
    def __init__(self, tipo, posx, velocidad,posy):
        self.id = next(Vehiculo.id_iter) #id
        self.t = tipo
        self.x = posx #X coordinate for location
        self.y = posy #Y coordinate
        self.vm = velocidad
        self.d = 0 #traveled distance
        self.a = 0.01 #aceleration default value
        self.m = random.choice(["vmax", "vm"]) #tag for following distance behavior, change the options considering the experiment you want to simulate
        self.c = random.choice(["vmax", "vm"]) #tag for lane change behavior, change the options considering the experiment you want to simulate
        self.f = vmax
        self.g = "ng"

    def mover(self,acel):
        self.a = acel
        self.vm = max(vmin, min(vmax, self.vm + self.a*pt))
        self.d =self.vm*pt

class Carril:
    def __init__(self, posy):
        self.y = posy

def parametros(v,vm):
  distancias = {
      ("vmax"): vmax,
      ("vm"): vm}
  return distancias.get((v),"NA")

def iniciar(n): #to create agents of the model
    global vehiculos, carriles, filenames

    filenames = []
    vehiculos=[]

    #to create lanes
    carriles = []
    c1 = Carril(1.5)
    carriles.append(c1)
    c2 = Carril(2)
    carriles.append(c2)

    #to create vehicles

    while len(vehiculos) < n:
        v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4), random.choice([c.y for c in carriles]))#,"vm")
        if round(v.x,0) not in [round(y.x,0) for y in vehiculos if v.y == y.y]:
            vehiculos.append(v)

def toroide(): #toroidal space
    for v in vehiculos:
        if v.x > tM:
            v.x = v.x % tM

def duplicar(): #to temporarily create a vehicle so the last vehicle in a lane can "see" the first one of the same lane.
    global a
    for c in carriles:
      if len([v.x for v in vehiculos if v.y == c.y])!= 0:
        for v in vehiculos:
          if v.x == min([v.x for v in vehiculos if v.y == c.y]):
            a = v.vm
            copia = Vehiculo("doble",min([v.x for v in vehiculos if v.y == c.y])+tM, a,c.y)
            vehiculos.append(copia)
      else:
          pass

def movimiento(): #to move vehicles along both lanes
  for c in carriles:
    vehiculos.sort(key = lambda v: v.x)
    for v,z in zip([v for v in vehiculos if v.y==c.y],[z for z in vehiculos if z.y==c.y][1:]):
      d = z.x - v.x #distance between vehicles
      v.f = parametros(v.m,(min(1,max(v.vm + 0.011,0.1))))
      if  d >= v.f*pt:
        v.mover(0.01+0.001*random.uniform(-1,1))
      else:
        v.vm = z.vm #to take the speed of the vehicle in front
        v.mover(-0.02) #to brake
        v.f = parametros(v.c, (min(1,max(v.vm + 0.011,0.1))))
        cruce(v,v.f,1)
      v.x = v.d + v.x


def cruce(v,fun,p): #for lane changing
    lmax=v.x+vmax
    lmin=v.x-fun
    for c in carriles:
      if len([z for z in vehiculos if z.y != v.y and z.x < lmax and z.x > lmin and abs(v.y-z.y)==abs(v.y-c.y)])==0 and v.y!=c.y:
        v.y = v.y - (v.y-c.y)
      else:
        pass

def borrar(): #to eliminate the temporarily vehicle
    for c in carriles:
        for v in vehiculos:
            if v.t == "doble":
                vehiculos.remove(v)

def parada(t,n,b): #to simulate suddenly stops
  for v in vehiculos:
    while len([v for v in vehiculos if v.g == "g" and v.t != "doble"]) <b*n and (t == 90 or t == 170) :
      g = random.choice([v for v in vehiculos if v.t != "doble"])
      g.g = "g"
      g.vm = 0

    if len([v for v in vehiculos if v.g == "g" and v.t != "doble"])!=0:
      v.g = "n.g"

def interaccion(n,s,b): #all functions are called here
    for i in range(1,t+1):
      duplicar()
      parada(i,n,b)
      movimiento()
      borrar()
      toroide()

      #Collected data from each step to generate graphics, we can change what we collect considering the experiment
      vel1.append([1,st.mean([v.vm for v in vehiculos]),n,s])
      med1.append([1,st.median([v.vm for v in vehiculos]),n,s])
      des1.append([1,st.stdev([v.vm for v in vehiculos]),n,s])
      err1.append([1,ss.sem([v.vm for v in vehiculos]),n,s])

      #Uncomment this part to create an image of each step and save them to be able to create a GIF:
      '''
      ig, ax = plt.subplots(figsize=(18,7))
      #carros
      #plot([v.x for v in vehiculos if v.m=="vmax"], [v.y for v in vehiculos if v.m=="vmax"] ,'o', color="darkorange", markersize=3)
      #plot([v.x for v in vehiculos if v.m=="vm"], [v.y for v in vehiculos if v.m=="vm"] ,'o', color="blue", markersize=3)
      #carriles
      ax.add_patch(patches.Rectangle((0, 0.8),50,0.4,edgecolor = 'gray',facecolor = 'gray',fill=True))
      text(1,0.17,"Paso:")
      text(2,0.2,i)
      tick_params(labelcolor='w')
      #grid(b=True, axis='both', color = "white")
      axis([0,tM,0.5,1.5])

      filename = f'paso{i}.png'
      filenames.append(filename)
      plt.savefig(filename)
      plt.close()
      #'''
      '''
      print(["Tiempo:",i])
      print(len([v for v in vehiculos if v.m=="vmax"]))
      print(len([v for v in vehiculos if v.m=="vm"]))
      '''




In [ ]:
#lists
vel1 = []
med1 = []
des1 = []
err1 = []


for v in Por:
  for i in N:
    for j in range (1,s+1): #el s es la cantidad de simulaciones
      iniciar(i)
      interaccion(i,j,v)
      media=[]
    media.append(st.mean([l[1] for l in vel1 if l[2]==i]))
    print(media)
    print("-----------------------------","Acabó N =",i,"-----------------------------")
  print("-----------------------------","Acabó Por =",v,"-----------------------------")


In [ ]:
#If the images were created, we can generate the GIF
gif_name= "Los 4 comportamientos"

with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        i = imageio.imread(filename)
        writer.append_data(i)

#This eliminates all the created images
for filename in set(filenames):
    os.remove(filename)

Con pista obstruída

In [29]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss
from datetime import datetime
import matplotlib.patches as patches

#Parameters
N =[10,20,30,40,50,60,70,80,90] #amount of vehicles (density)
pt = 1 #unit time (step)
s = 100 #number of simulations
t = 500 #time that the each simulation longs
vmax = 1 #speed limit
vmin= 0 #speed limit
tM= 50 #capacity for each lane

class Vehiculo:
    id_iter = itertools.count() #to generate an id for each agent
    def __init__(self, tipo, posx, velocidad,posy):
        self.id = next(Vehiculo.id_iter) #id
        self.t = tipo
        self.x = posx #X coordinate for location
        self.y = posy #Y coordinate
        self.vm = velocidad
        self.d = 0 #traveled distance
        self.a = 0.01 #aceleration default value
        self.m = random.choice(["vmax", "vm"]) #tag for following distance behavior, change the options considering the experiment you want to simulate
        self.c = random.choice(["vmax", "vm"]) #tag for lane change behavior, change the options considering the experiment you want to simulate
        self.f = vmax

    def mover(self,acel):
        self.a = acel
        self.vm = max(vmin, min(vmax, self.vm + self.a*pt))
        self.d =self.vm*pt

class Carril:
    def __init__(self, posy):
        self.y = posy

def parametros(v,vm):
  distancias = {
      ("vmax"): vmax,
      ("vm"): vm}
  return distancias.get((v),"NA")

def iniciar(n): #to create agents of the model
    global vehiculos, carriles, filenames

    filenames = []
    vehiculos=[]

    #to create lanes
    carriles = []
    c1 = Carril(1.5)
    carriles.append(c1)
    c2 = Carril(2)
    carriles.append(c2)

    #to create vehicles

    while len(vehiculos) < n:
        v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4), random.choice([c.y for c in carriles]))#,"vm")
        if round(v.x,0) not in [round(y.x,0) for y in vehiculos if v.y == y.y]:
            vehiculos.append(v)

def toroide(): #toroidal space
    for v in vehiculos:
        if v.x > tM:
            v.x = v.x % tM

def duplicar(): #to temporarily create a vehicle so the last vehicle in a lane can "see" the first one of the same lane.
    global a
    for c in carriles:
      if len([v.x for v in vehiculos if v.y == c.y])!= 0:
        for v in vehiculos:
          if v.x == min([v.x for v in vehiculos if v.y == c.y]):
            a = v.vm
            copia = Vehiculo("doble",min([v.x for v in vehiculos if v.y == c.y])+tM, a,c.y)
            vehiculos.append(copia)
      else:
          pass

def movimientoconobst(): #movement considering an obstruction on the road
    for c in carriles:
      vehiculos.sort(key = lambda v: v.x)
      for v,z in zip([v for v in vehiculos if v.y==c.y],[z for z in vehiculos if z.y==c.y][1:]):
          d = z.x - v.x
          if v.x + 1> 20 and v.y==1.5 and v.x < 30: #if the vehicle is near to the obstruction, stops and tries to change lane
            v.vm=0
            v.f = parametros(v.c, (min(1,max(v.vm + 0.011,0.1))))
            cruce2(v,v.f)
          else: #if not, normal situation
            v.f = parametros(v.m, (min(1,max(v.vm + 0.011,0.1))))
            if  d >= v.f*pt:
                v.mover(0.01+0.001*random.uniform(-1,1))
            else:
                v.vm = z.vm
                v.mover(-0.02)
                if v.x + 1> 20 and v.x <30 and v.y==2: #but if the vehicle is in the other lane but at the same level of the obstruction, doesn't try to change lane
                  pass
                else:
                  v.f = parametros(v.c, (min(1,max(v.vm + 0.011,0.1))))
                  cruce(v,v.f,1)
            v.x = v.d + v.x


def cruce(v,fun,p): #for lane changing
    lmax=v.x+vmax
    lmin=v.x-fun
    for c in carriles:
      if len([z for z in vehiculos if z.y != v.y and z.x < lmax and z.x > lmin and abs(v.y-z.y)==abs(v.y-c.y)])==0 and v.y!=c.y:
        v.y = v.y - (v.y-c.y)
      else:
        pass

def cruce2(v,fun):
    lmax=v.x+vmax
    lmin=v.x-fun
    if len([z for z in vehiculos if z.y != v.y and z.x < lmax and z.x > lmin])==0:
      v.y=2
    else:
      pass

def borrar(): #to eliminate the temporarily vehicle
    for c in carriles:
        for v in vehiculos:
            if v.t == "doble":
                vehiculos.remove(v)

def interaccion(n,s): #all functions are called here
    for i in range(1,t+1):
      duplicar()
      movimientoconobst()
      borrar()
      toroide()

      #Collected data from each step to generate graphics, we can change what we collect considering the experiment
      vel1.append([1,st.mean([v.vm for v in vehiculos]),n,s])
      med1.append([1,st.median([v.vm for v in vehiculos]),n,s])
      des1.append([1,st.stdev([v.vm for v in vehiculos]),n,s])
      err1.append([1,ss.sem([v.vm for v in vehiculos]),n,s])

      #Uncomment this part to create an image of each step and save them to be able to create a GIF:
      '''
      ig, ax = plt.subplots(figsize=(18,7))
      #carros
      #plot([v.x for v in vehiculos if v.m=="vmax"], [v.y for v in vehiculos if v.m=="vmax"] ,'o', color="darkorange", markersize=3)
      #plot([v.x for v in vehiculos if v.m=="vm"], [v.y for v in vehiculos if v.m=="vm"] ,'o', color="blue", markersize=3)
      #carriles
      ax.add_patch(patches.Rectangle((0, 0.8),50,0.4,edgecolor = 'gray',facecolor = 'gray',fill=True))
      text(1,0.17,"Paso:")
      text(2,0.2,i)
      tick_params(labelcolor='w')
      #grid(b=True, axis='both', color = "white")
      axis([0,tM,0.5,1.5])

      filename = f'paso{i}.png'
      filenames.append(filename)
      plt.savefig(filename)
      plt.close()
      #'''
      '''
      print(["Tiempo:",i])
      print(len([v for v in vehiculos if v.m=="vmax"]))
      print(len([v for v in vehiculos if v.m=="vm"]))
      '''




In [ ]:
#lists
vel1 = []
med1 = []
des1 = []
err1 = []

for i in N:
  for j in range (1,s+1): #el s es la cantidad de simulaciones
    iniciar(i)
    interaccion(i,j)
    media=[]
  media.append(st.mean([l[1] for l in vel1 if l[2]==i]))
  print(media)
  print("-----------------------------","Acabó N =",i,"-----------------------------")

[0.6076418044563819]
----------------------------- Acabó N = 10 -----------------------------
[0.39480935942808826]
----------------------------- Acabó N = 20 -----------------------------
[0.2887811127272323]
----------------------------- Acabó N = 30 -----------------------------
[0.2227959381003202]
----------------------------- Acabó N = 40 -----------------------------
[0.18825391115570736]
----------------------------- Acabó N = 50 -----------------------------


#Type A, B and C

In [8]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss
import matplotlib.patches as patches

#Parameters
N =[10,20,30,40,50,60,70,80,90]
pt = 1
s = 100
t = 500
vemax = 1
vemin= 0
tM= 50

class Vehiculo:
    id_iter = itertools.count()
    def __init__(self, tipo, posx, velocidad,posy,mov,cru):
        self.id = next(Vehiculo.id_iter) #id
        self.t = tipo
        self.x = posx
        self.y = posy
        self.vm = velocidad
        self.d = 0
        self.a = 0.01
        self.m = mov
        self.c = cru
        self.f = vemax
        self.e= "libre"
        self.temp=0

    def mover(self,acel):
        if self.t == "coaster":
            vmax = 2
        else:
            vmax = 1

        self.a = acel
        self.vm = max(vemin, min(vmax, self.vm + self.a*pt))
        self.d =self.vm*pt

class Carril:
    def __init__(self, posy):
        self.y = posy

def iniciar(n): #to create agents
    global vehiculos, carriles, filenames, coas, com


    filenames = []

    #to create lanes
    carriles = []
    c1 = Carril(0.75)
    carriles.append(c1)
    c2 = Carril(1.25)
    carriles.append(c2)

    #to create vehicles
    vehiculos = []
    while len(vehiculos) < n: #change random.choice(["coaster","carro","combi"]) considering the types in the experiment
        v = Vehiculo(random.choice(["carro","coaster"]), round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4), random.choice([c.y for c in carriles]), random.choice(["vemax","vm"]), random.choice(["vemax", "vm"]))
        if round(v.x,0) not in [round(y.x,0) for y in vehiculos if v.y == y.y]:
            vehiculos.append(v)

    #uncomment this part of the code for the experiment where Type B and C stop:
    coas=[]
    #while len(coas) < 4:
      #coas.append(random.choice([v.id for v in vehiculos if v.t=="coaster"]))

    com=[]
    #while len(com) < 16:
      #com.append(random.choice([v.id for v in vehiculos if v.t=="combi"]))

def parametros(v,vm):
  distancias = {
      ("vemax"): vemax,
      ("vm"): vm}
  return distancias.get((v),"NA")


def toroide(): #toroidal space
    for v in vehiculos:
        if v.x > tM:
            v.x = v.x % tM

def duplicar():
  global a
  for c in carriles:
    if len([v.x for v in vehiculos if v.y == c.y])!= 0:
        for v in vehiculos:
          if v.x == min([v.x for v in vehiculos if v.y == c.y]):
            a = v.vm
            copia = Vehiculo("doble",min([v.x for v in vehiculos if v.y == c.y])+tM, a,c.y,"vemax","vemax")
            vehiculos.append(copia)
    else:
        pass


def movimiento():
    for c in carriles:
      vehiculos.sort(key = lambda v: v.x)
      for v,z in zip([v for v in vehiculos if v.y==c.y],[z for z in vehiculos if z.y==c.y][1:]):
          d = z.x - v.x #distance between vehicles
          if v.e == "libre":
            freno()
            if  d >= v.f*pt:
              if v.t == "combi":
                  v.mover(0.02+0.002*random.uniform(-1,1))
              else:
                  v.mover(0.01+0.001*random.uniform(-1,1))
            else:
              v.vm = z.vm #takes the speed of the vehicle in front
              v.mover(-0.02) #brakes
              cambio()
              cruce(v,v.f)
            v.x = v.d + v.x


def cruce(v,fun):
    lmax=v.x+vemax
    lmin=v.x-fun
    for c in carriles:
      if len([z for z in vehiculos if z.y != v.y and z.x < lmax and z.x > lmin and abs(v.y-z.y)==abs(v.y-c.y)])==0 and v.y!=c.y:
        v.y = v.y - (v.y-c.y)
      else:
        pass

def freno():
    for v in vehiculos:
        if v.t == "coaster":
            v.f = min(2,max(v.vm + 0.011,0.1))
        elif v.t == "combi":
            v.f = min(1,max(v.vm + 0.022,0.1))
        else:
            v.f = parametros(v.m, min(1,max(v.vm + 0.011,0.1)))

def cambio():
    for v in vehiculos:
        if v.t == "coaster":
            v.f = min(2,max(v.vm + 0.011,0.1))
        elif v.t == "combi":
            v.f = min(1,max(v.vm + 0.022,0.1))
        else:
            v.f = parametros(v.c, min(1,max(v.vm + 0.011,0.1)))

def parada(t):
    for v in vehiculos:
      if v.t == "coaster" and v.e=="libre" and t%100==0 and coas.__contains__(v.id) and len([v for v in vehiculos if v.e=="pare" and v.t=="coaster"])<1:
        v.vm = 0
        v.e="pare"
        v.temp=t

      if v.t == "combi" and v.e=="libre" and t%30==0 and com.__contains__(v.id) and len([v for v in vehiculos if v.e=="pare" and v.t=="combi"])<1:
        v.vm = 0
        v.e="pare"
        v.temp=t


def avanzar(t): #vehicles restart moving after the stop
    for v in vehiculos:
        if v.t == "coaster" and v.e=="pare" and t-v.temp==20:
            v.e = "libre"

        if v.t == "combi" and v.e=="pare" and t-v.temp==5:
            v.e = "libre"

def borrar():
    for c in carriles:
        for v in vehiculos:
            if v.t == "doble":
                vehiculos.remove(v)


def interaccion(n,s):

  for i in range(1,t+1):
    #parada(i) #uncomment this function of the code for the stop experiment
    #avanzar(i) #uncomment this function of the code for the stop experiment
    duplicar()
    movimiento()
    borrar()
    toroide()

    #Collected data from each step to generate graphics, we can change what we collect considering the experiment
    vel1.append([1,st.mean([v.vm for v in vehiculos]),n,s])
    med1.append([1,st.median([v.vm for v in vehiculos]),n,s])
    des1.append([1,st.stdev([v.vm for v in vehiculos]),n,s])
    err1.append([1,ss.sem([v.vm for v in vehiculos]),n,s])

    #Uncomment this part to create an image of each step and save them to be able to create a GIF:
    '''
    fig, ax = plt.subplots(figsize=(20,2))
    plot([v.x for v in vehiculos if v.t == "carro"], [v.y for v in vehiculos if v.t == "carro"] ,'bs')
    plot([v.x for v in vehiculos if v.t == "coaster" and v.e=="pare"], [v.y for v in vehiculos if v.t == "coaster" and v.e=="pare"] ,'r>')
    plot([v.x for v in vehiculos if v.t == "coaster" and v.e=="libre" ], [v.y for v in vehiculos if v.t == "coaster" and v.e=="libre"] ,'g>')
    plot([v.x for v in vehiculos if v.t == "combi" and v.e=="pare"], [v.y for v in vehiculos if v.t == "combi" and v.e=="pare"] ,'ro')
    plot([v.x for v in vehiculos if v.t == "combi" and v.e=="libre" ], [v.y for v in vehiculos if v.t == "combi" and v.e=="libre"] ,'ko')
    ax.add_patch(patches.Rectangle((0, 0.5),50,0.5,edgecolor = 'white',facecolor = 'gray',fill=True))
    ax.add_patch(patches.Rectangle((0, 1),50,0.5,edgecolor = 'white',facecolor = 'gray',fill=True))
    text(tM/50,1.8,"Paso:")
    text(tM/20,1.8,i)
    axis([0,50,0,2])
    filename = f'paso{i}.png'
    filenames.append(filename)
    plt.savefig(filename)
    plt.close()
    #'''





In [ ]:
#list
vel1 = []
med1 = []
des1 = []
err1 = []

for i in N:
  for j in range (1,s+1): #s is the number of times that the simulation is iterated
    iniciar(i)
    interaccion(i,j)
  media=[]
  media.append(st.mean([l[1] for l in vel1 if l[2]==i]))
  print(media)
  print("-----------------------------","Acabó N =",i,"-----------------------------")

In [ ]:
#If the images were created, we can generate the GIF
gif_name= "Los 4 comportamientos"

with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        i = imageio.imread(filename)
        writer.append_data(i)

#This eliminates all the created images
for filename in set(filenames):
    os.remove(filename)